In [524]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [525]:
# Read the data
sales_predict_df = pd.read_csv('./Resources/Train_Output_CSV.csv')
sales_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [526]:
#Drop the target column "Item_Outlet_Sales" fro the feature DataFrame
features_df = sales_predict_df.drop(columns=['Item_Outlet_Sales','Item_Identifier'])




In [527]:
# One-hot encoding using pd.get_dummies
features_df = pd.get_dummies(features_df, columns=['Outlet_Size' ,'Item_Type' ,'Outlet_Location_Type','Outlet_Establishment_Year','Outlet_Identifier','Outlet_Type', 'Item_Fat_Content','Item_Weight'])

# Extract target variables
target_df = sales_predict_df['Item_Outlet_Sales']
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Columns: 465 entries, Item_Visibility to Item_Weight_21.35
dtypes: float64(2), uint8(463)
memory usage: 3.9 MB


In [528]:
# Convert to numpy arrays
X = features_df.values
y = target_df.values


In [529]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [530]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [531]:
print("Any missing in X_train_scaled:", np.isnan(X_train_scaled).any())
print("Any missing in y_train:", np.isnan(y_train).any())


Any missing in X_train_scaled: False
Any missing in y_train: False


In [532]:

# mask = ~np.isnan(y_train)
# X_train_scaled = X_train_scaled[mask]
# y_train = y_train[mask]


In [533]:

# imputer = SimpleImputer(strategy="mean")
# y_train = imputer.fit_transform(y_train.reshape(-1, 1)).ravel()


## refrence 
# Parameter Tuning: A Complete Guide with Python Codes
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/#h-general-parameters

In [534]:
# Uee XGBoost to define regressor model
regressor = XGBRegressor(learning_rate=0.013420,
    n_estimators=430,
    max_depth=3,
    min_child_weight=23,
    gamma=4,
    subsample=.9,
    colsample_bytree=.65)

# Fit the model to the training data
regressor.fit(X_train_scaled, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.65, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=4, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01342, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=23, missing=nan, monotone_constraints=None,
             n_estimators=430, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [535]:
#Running the model on the training data to predict sales
sales_data_predictions = regressor.predict(X_test_scaled)


In [536]:

# Re-run the prediction on modified X_test_scaled
sales_data_predictions = regressor.predict(X_test_scaled)

# Calculate the R-squared value again
r2_sales = metrics.r2_score(y_test, sales_data_predictions)
print('R Squared value = ', r2_sales)


R Squared value =  0.6247218515656274
